# 다중 회귀(multiple Regression)

여러 개의 특성을 이용하는 회귀 모델

* `feature engineering`: 기존 feature을 이용해 새로운 feature을 만드는 것.

In [10]:
import pandas as pd

df = pd.read_csv('https://bit.ly/perch_csv_data')
perch_full = df.to_numpy() # 넘파이 배열로 변환
perch_full[:5]

array([[ 8.4 ,  2.11,  1.41],
       [13.7 ,  3.53,  2.  ],
       [15.  ,  3.82,  2.43],
       [16.2 ,  4.59,  2.63],
       [17.4 ,  4.59,  2.94]])

In [11]:
import numpy as np

perch_weight = np.array([5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 110.0,
       115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 130.0,
       150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 197.0,
       218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 514.0,
       556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 820.0,
       850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 1000.0,
       1000.0])

In [16]:
print(perch_weight.shape, perch_full.shape)

(56,) (56, 3)


In [24]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(perch_full,
                                                                     perch_weight,
                                                                     random_state = 42)

* `transform()`: 사이킷런에서 제공하는 변환기


2개의 원소의 특징을 가지고 있는 샘플 [2,3]이 6개의 특성을 가진 [1. 2. 3. 4. 6. 9.]샘플이 되었다.

각 특성의 제곱항과 서로를 곱한 항을 추가한 것. 이때 1이 추가되는데 이는 선형 방정식의 절편에 곱해지는 값으로 선형모델에서는 자동으로 절편을 추가하므로 `include_bias = False`로 지정하여 trasform에서 빼줄 수 있다

In [5]:
from sklearn.preprocessing import PolynomialFeatures

In [6]:
poly = PolynomialFeatures()

poly.fit([[2,3]])
print(poly.transform([[2,3]]))

[[1. 2. 3. 4. 6. 9.]]


In [7]:
poly = PolynomialFeatures(include_bias = False)

poly.fit([[2,3]])
print(poly.transform([[2,3]]))

[[2. 3. 4. 6. 9.]]


In [14]:
poly = PolynomialFeatures(include_bias=False)

poly.fit(train_input)
train_poly = poly.transform(train_input)
print(train_poly.shape)

(42, 9)


In [18]:
poly.get_feature_names_out() # 각 특성이 어떤 입력 조합으로 만들어줬는지 알려줌

array(['x0', 'x1', 'x2', 'x0^2', 'x0 x1', 'x0 x2', 'x1^2', 'x1 x2',
       'x2^2'], dtype=object)

In [25]:
test_poly = poly.transform(test_input)

In [20]:
from sklearn.linear_model import LinearRegression

In [27]:
lr = LinearRegression()

lr.fit(train_poly, train_target)
print("훈련 데이터에 대한 예측 정확도: ",lr.score(train_poly, train_target))
print("테스트 데이터에 대한 예측 정확도: ",lr.score(test_poly, test_target))

훈련 데이터에 대한 예측 정확도:  0.9903183436982126
테스트 데이터에 대한 예측 정확도:  0.9714559911594143


특성을 추가하여(두께) 사용하고 특성 공학(feature engineering)을 통해 특성을 추가하였더니 `선형 회귀 능력이 매우 강해지는 것`을 확인할 수 있다. 


* 이때 3제곱항, 4제곱항을 한 특성을 더 추가하면?

다음 degree =5 로 설정하여 5제곱항까지 특성에 추가되도록 해보자.


In [30]:
poly = PolynomialFeatures(degree =5, include_bias = False)

poly.fit(train_input)
train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input)

train_poly.shape # 특성의 개수가 55개

(42, 55)

In [32]:
lr.fit(train_poly,train_target)
lr.score(train_poly, train_target)


0.999999999998714

In [33]:
lr.score(test_poly,test_target)

-144.4066764700865

훈련 데이터에 관해서는 선형 모델이 매우 강력해지지만 과대적합 문제가 발생할 수 있다.

# 규제

모델이 훈련 데이터를 너무 과도하게 학습하지 못하도록 규제하는 것을 말한다.선형 회귀 모델의 경우 특성에 곱해지는 계수(기울기) 크기를 작게하는 것을 말한다.